<a href="https://colab.research.google.com/github/Ragavi203/AI-Mental-Health-Chatbot/blob/main/AI_Mental_Health_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install textblob

In [2]:
# Import necessary libraries
import re
from textblob import TextBlob
import random
import warnings
import datetime

In [12]:
import re
from textblob import TextBlob
import random
import datetime
from collections import deque

class EnhancedMentalHealthBot:
    def __init__(self):
        # Initialize conversation memory with a deque to maintain recent context
        self.conversation_history = deque(maxlen=5)  # Keep last 5 exchanges for context
        self.user_state = {
            'current_emotion': None,
            'emotion_intensity': None,
            'previous_emotions': [],
            'identified_topics': set(),
            'coping_strategies_used': set(),
            'session_start': datetime.datetime.now()
        }

        # Enhanced emotion detection
        self.emotion_patterns = {
            'anxiety': r'\b(anxious|worried|nervous|stressed|overwhelmed|panic)\b',
            'depression': r'\b(depressed|sad|hopeless|worthless|empty|lonely)\b',
            'anger': r'\b(angry|frustrated|mad|irritated|upset|furious)\b',
            'fear': r'\b(scared|afraid|fearful|terrified|worried|dread)\b',
            'joy': r'\b(happy|excited|pleased|grateful|content|peaceful)\b'
        }

        # Topic identification patterns
        self.topic_patterns = {
            'work': r'\b(work|job|career|boss|colleague|workplace)\b',
            'relationships': r'\b(relationship|partner|friend|family|parent|child)\b',
            'health': r'\b(health|sick|pain|doctor|symptoms|illness)\b',
            'finance': r'\b(money|financial|debt|bills|expenses|budget)\b',
            'education': r'\b(school|study|exam|class|grade|homework)\b'
        }

        # Enhanced response templates with more context-aware options
        self.responses = {
            'anxiety': {
                'high': [
                    "I can hear how overwhelming this anxiety feels. Let's take a moment to ground ourselves. Would you like to try a breathing exercise together?",
                    "When anxiety is this intense, it can feel all-consuming. Remember that you've gotten through difficult moments before. What has helped you manage anxiety in the past?"
                ],
                'medium': [
                    "It sounds like you're experiencing some anxiety. Would you like to explore what might be triggering these feelings?",
                    "Anxiety can be challenging to navigate. Should we break down what you're feeling into smaller, more manageable parts?"
                ],
                'low': [
                    "I notice some anxious feelings in what you're sharing. Would you like to talk more about what's on your mind?",
                    "Sometimes naming our anxious feelings can help us understand them better. What does this anxiety feel like for you?"
                ]
            },
            'depression': {
                'high': [
                    "I hear how heavy these feelings are. While I'm here to listen, I want to make sure you have support. Would you like information about professional resources?",
                    "Depression can make everything feel overwhelming. Let's focus on one small step at a time. What's one tiny thing that might help right now?"
                ],
                'medium': [
                    "It sounds like you're going through a difficult time. What kind of support would be most helpful right now?",
                    "Depression can make us feel very alone. Would you like to explore some ways to connect with support systems?"
                ],
                'low': [
                    "I notice some sadness in what you're sharing. Would you like to talk more about what's contributing to these feelings?",
                    "Sometimes low moods can sneak up on us. What changes have you noticed in your daily life recently?"
                ]
            }
            # Additional emotion categories and responses can be added here
        }

        # Expanded coping strategies with categories
        self.coping_strategies = {
            'anxiety': [
                "Try box breathing: Inhale for 4 counts, hold for 4, exhale for 4, hold for 4. Repeat 4 times.",
                "Use the 5-4-3-2-1 grounding technique: Name 5 things you see, 4 you feel, 3 you hear, 2 you smell, and 1 you taste.",
                "Progressive muscle relaxation: Tense and relax each muscle group, starting from your toes and moving up."
            ],
            'depression': [
                "Try to do one small enjoyable activity, even if it's just for 5 minutes.",
                "Use the 'opposite action' technique: When you feel like isolating, try reaching out to one person.",
                "Practice self-compassion by talking to yourself as you would to a good friend."
            ]
        }

    def analyze_emotion(self, text):
        """
        Performs detailed emotion analysis using multiple methods:
        1. Pattern matching for specific emotion words
        2. Sentiment analysis for intensity
        3. Context consideration from conversation history
        """
        # Initialize emotion scores
        emotion_scores = {emotion: 0 for emotion in self.emotion_patterns}

        # Pattern matching for emotions
        for emotion, pattern in self.emotion_patterns.items():
            matches = len(re.findall(pattern, text.lower()))
            emotion_scores[emotion] += matches

        # Sentiment analysis for intensity
        sentiment = TextBlob(text).sentiment.polarity
        intensity = abs(sentiment)  # Use absolute value for intensity

        # Determine primary emotion
        primary_emotion = max(emotion_scores.items(), key=lambda x: x[1])[0]
        if all(score == 0 for score in emotion_scores.values()):
            primary_emotion = 'neutral'

        # Classify intensity
        if intensity > 0.7:
            intensity_level = 'high'
        elif intensity > 0.3:
            intensity_level = 'medium'
        else:
            intensity_level = 'low'

        return primary_emotion, intensity_level

    def identify_topics(self, text):
        """
        Identifies discussion topics from user input.
        """
        topics = set()
        for topic, pattern in self.topic_patterns.items():
            if re.search(pattern, text.lower()):
                topics.add(topic)
        return topics

    def update_user_state(self, text):
        """
        Updates the user's emotional state and context tracking.
        """
        emotion, intensity = self.analyze_emotion(text)
        topics = self.identify_topics(text)

        self.user_state['current_emotion'] = emotion
        self.user_state['emotion_intensity'] = intensity
        self.user_state['previous_emotions'].append(emotion)
        self.user_state['identified_topics'].update(topics)

    def generate_response(self, user_input):
        """
        Generates a context-aware response based on emotion analysis,
        conversation history, and identified topics.
        """
        # Update user state with new input
        self.update_user_state(user_input)

        # Store conversation for context
        self.conversation_history.append({
            'user_input': user_input,
            'timestamp': datetime.datetime.now(),
            'emotion': self.user_state['current_emotion'],
            'intensity': self.user_state['emotion_intensity']
        })

        # Check for crisis keywords first
        if self.detect_crisis_keywords(user_input):
            return self.get_crisis_response()

        # Generate contextual response
        emotion = self.user_state['current_emotion']
        intensity = self.user_state['emotion_intensity']

        # If we have specific responses for this emotion
        if emotion in self.responses and intensity in self.responses[emotion]:
            response = random.choice(self.responses[emotion][intensity])
        else:
            # Default supportive response if no specific match
            response = "I hear what you're saying. Would you like to tell me more about how you're feeling?"

        # Add relevant coping strategy if appropriate
        if emotion in self.coping_strategies and random.random() < 0.3:  # 30% chance
            strategy = random.choice(self.coping_strategies[emotion])
            response += f"\n\nHere's a technique that might help: {strategy}"

        return response

    def detect_crisis_keywords(self, text):
        """
        Enhanced crisis detection with broader keyword recognition
        and pattern matching.
        """
        crisis_patterns = [
            r'\b(suicide|kill|harm|hurt)\s*(myself|self)\b',
            r'\b(want|wish)\s*to\s*(die|end it|be dead)\b',
            r'\bno\s*(reason|point)\s*to\s*(live|go on)\b',
            r'\b(can\'t|cannot|don\'t want to)\s*(go on|continue|live)\b'
        ]

        return any(re.search(pattern, text.lower()) for pattern in crisis_patterns)

    def get_crisis_response(self):
        """
        Enhanced crisis response with more supportive language
        and clear action steps.
        """
        response = [
            "I'm very concerned about what you're sharing and I want you to know that you're not alone. ",
            "While I'm here to listen, it's important that you speak with someone who can provide proper support. ",
            "\nHere are some immediate resources that can help:\n\n"
        ]

        resources = {
            "24/7 Crisis Hotline": "988",
            "Crisis Text Line": "Text HOME to 741741",
            "Emergency Services": "911",
            "Online Crisis Chat": "https://suicidepreventionlifeline.org/chat/"
        }

        for service, contact in resources.items():
            response.append(f"• {service}: {contact}\n")

        response.append("\nWould you be willing to reach out to one of these services? "
                       "They have trained professionals who want to help and understand "
                       "what you're going through.")

        return "".join(response)

    def suggest_coping_strategy(self):
        """
        Returns a contextually appropriate coping strategy based on
        the user's current emotional state and history.
        """
        emotion = self.user_state['current_emotion']
        if emotion in self.coping_strategies:
            unused_strategies = [s for s in self.coping_strategies[emotion]
                               if s not in self.user_state['coping_strategies_used']]
            if unused_strategies:
                strategy = random.choice(unused_strategies)
                self.user_state['coping_strategies_used'].add(strategy)
                return strategy

        # Default strategy if no emotion-specific ones are available
        return random.choice(self.coping_strategies['anxiety'])  # Using anxiety strategies as default

In [13]:
# Create the enhanced bot
bot = EnhancedMentalHealthBot()

# Function for interaction
def chat_with_bot(message):
    print("\nYou:", message)
    response = bot.generate_response(message)
    print("\nSupport Bot:", response)

# Example usage
chat_with_bot("I'm feeling really anxious about my upcoming presentation at work")


You: I'm feeling really anxious about my upcoming presentation at work

Support Bot: I notice some anxious feelings in what you're sharing. Would you like to talk more about what's on your mind?

Here's a technique that might help: Use the 5-4-3-2-1 grounding technique: Name 5 things you see, 4 you feel, 3 you hear, 2 you smell, and 1 you taste.
